## 数据处理

In [ ]:
# 切分数据 按照章节进行切分 其实并不通用
import re

def is_chapter(line):
    pattern = r'第.*章$'

i = 0
file_name = 'chapter' + str(i) + '.txt'

pre_text = ''

with open('./test.txt') as f:
    for line in f.readlines():
        if is_chapter(line[0:3]):
            i += 1
            file_name = 'chapter' + str(i) + '.txt'
            line = pre_text + '\n' +line

        with open('./chapter' + file_name,'a') as f_chapter:
            f_chapter.write(line)

## 构建本地数据库

In [4]:
# 设置矢量数据库
# 要创建本地 Milvus 矢量数据库，只需MilvusClient通过指定一个文件名来实例化一个来存储所有数据，例如“milvus_demo.db”。

from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

# 生成了.milvus_demo.db.lock 和 milvus_demo.db


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
# 创建收藏夹
# 在 Milvus 中，我们需要一个集合来存储向量及其相关元数据。你可以把它想象成传统 SQL 数据库中的表。
# 创建集合时，你可以定义 schema 和 index 参数来配置向量规范，例如维度、索引类型和距离度量。还有一些复杂的概念可以优化索引以提高向量搜索性能。
# 现在，我们只关注基础知识，并尽可能使用默认值。至少，你只需要设置集合名称和集合的向量字段的维度。

if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)


In [16]:
import os
file_path = '../../data/chunks'
def create_docs_from_txt(txt_file):
    doc = []
    for txt_file in os.listdir(file_path):
        if txt_file.endswith('.txt'):  # 确保只处理 .txt 文件
            sentence = ''
            with open(os.path.join(file_path, txt_file), "r") as f:

                for line in f.readlines():
                    sentence += str(line[:-1]) # :-1 是为了去掉每行末尾的换行符
                    # if len(sentence) > 500:
                    #     # 如果chunk_size大于embedding模型的max_token数
                        
                    #     sentence += '。'
                        # break
            doc.append(sentence)

    return doc

documents = create_docs_from_txt(file_path)

for line in documents:
    print(len(line))

1030
906
958
1002
847
983
1019
1027


## 加载embedding模型

In [13]:
import os

os.path.abspath('.')

'/home/cdjzxy/dev_llm/ChatBot_Gradio/版本三/Milvus'

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('../../bce-embedding-base_v1')

In [4]:
from sentence_transformers import SentenceTransformer
queries = ['query_1', 'query_2']
passages = ["样例文档-1", "样例文档-2"]
instruction = "为这个句子生成表示以用于检索相关文章："

model = SentenceTransformer('../../bce-embedding-base_v1')
q_embeddings = model.encode([instruction+q for q in queries], normalize_embeddings=True)
p_embeddings = model.encode(passages, normalize_embeddings=True)
scores = q_embeddings @ p_embeddings.T
scores

array([[0.42265287, 0.34486026],
       [0.35691443, 0.4063045 ]], dtype=float32)

In [14]:
q_embeddings

array([[-0.01750912, -0.03124577,  0.00544266, ...,  0.02072966,
         0.03551982, -0.00630776],
       [-0.00536352, -0.03788967, -0.0096715 , ...,  0.03025958,
         0.02711708, -0.02719999]], dtype=float32)

In [6]:
from sentence_transformers import SentenceTransformer
queries = ['query_1', 'query_2']
passages = ["样例文档-1", "样例文档-2"]
instruction = "为这个句子生成表示以用于检索相关文章："

model = SentenceTransformer('../../nlp_gte_sentence-embedding_chinese-large') # 中文通用领域下载量最多
q_embeddings = model.encode([instruction+q for q in queries], normalize_embeddings=True)
p_embeddings = model.encode(passages, normalize_embeddings=True)
scores = q_embeddings @ p_embeddings.T
scores

No sentence-transformers model found with name ../../nlp_gte_sentence-embedding_chinese-large. Creating a new one with mean pooling.


array([[0.4102361 , 0.2932835 ],
       [0.2845833 , 0.38754874]], dtype=float32)

In [ ]:
from sentence_transformers import SentenceTransformer
queries = ['query_1', 'query_2']
passages = ["样例文档-1", "样例文档-2"]
instruction = "为这个句子生成表示以用于检索相关文章："

model = SentenceTransformer('../../bge-large-zh') # 司马排名第一哦
q_embeddings = model.encode([instruction+q for q in queries], normalize_embeddings=True)
p_embeddings = model.encode(passages, normalize_embeddings=True)
scores = q_embeddings @ p_embeddings.T

## 使用milvus自带的embedding模型

In [17]:
# 准备数据

# 用向量表示文本
# 首先，安装模型库。此软件包包含 PyTorch 等基本 ML 工具。如果您的本地环境从未安装过 PyTorch，则软件包下载可能需要一些时间。

from pymilvus import model


embedding_fn = model.DefaultEmbeddingFunction() # 默认的embeding模型 效果待评估 之前用过bge-4
 

docs = create_docs_from_txt(file_path)
vectors = embedding_fn.encode_documents(documents)

# print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]
data
# print("Data has", len(data), "entities, each with fields: ", data[0].keys())
# print("Vector dim:", len(data[0]["vector"]))


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like GPTCache/paraphrase-albert-small-v2 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

## 插入数据

In [6]:
res = client.insert(collection_name="demo_collection", data=data)

print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


In [8]:
# 语义搜索
# 现在我们可以通过将搜索查询文本表示为向量来进行语义搜索，并在 Milvus 上进行向量相似性搜索。

# 向量搜索
# Milvus 可以同时接受一个或多个向量搜索请求。query_vectors 变量的值是一个向量列表，其中每个向量都是一个浮点数数组。

query_vectors = embedding_fn.encode_queries(["象"])

res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)


data: ["[{'id': 2, 'distance': 1.0, 'entity': {'text': '大象', 'subject': 'history'}}, {'id': 0, 'distance': 1.0, 'entity': {'text': '老虎', 'subject': 'history'}}]"] 


In [9]:
# 带元数据过滤的向量搜索
# 您还可以在考虑元数据的值（在 Milvus 中称为“标量”字段，因为标量指的是非向量数据）的同时进行向量搜索。
# 这可以通过指定某些条件的过滤表达式来完成。让我们subject在以下示例中看看如何使用字段进行搜索和过滤。


docs = [
    "Machine learning has been used for drug design.",
    "Computational synthesis with AI algorithms predicts molecular properties.",
    "DDR1 is involved in cancers and fibrosis.",
]
vectors = embedding_fn.encode_documents(docs)
data = [
    {"id": 3 + i, "vector": vectors[i], "text": docs[i], "subject": "biology"}
    for i in range(len(vectors))
]

client.insert(collection_name="demo_collection", data=data)

res = client.search(
    collection_name="demo_collection",
    data=embedding_fn.encode_queries(["tell me AI related information"]),
    filter="subject == 'biology'",
    limit=2,
    output_fields=["text", "subject"],
)

print(res)


data: ["[{'id': 4, 'distance': 0.2703056335449219, 'entity': {'text': 'Computational synthesis with AI algorithms predicts molecular properties.', 'subject': 'biology'}}, {'id': 3, 'distance': 0.1642589271068573, 'entity': {'text': 'Machine learning has been used for drug design.', 'subject': 'biology'}}]"] 


## 读取数据

In [1]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")


In [3]:
res = client.query(
    collection_name="demo_collection",
    filter="subject == 'history'",
    output_fields=["text", "subject"],
)
res

data: ["{'id': 0, 'text': '老虎', 'subject': 'history'}", "{'id': 1, 'text': 'A狮子', 'subject': 'history'}", "{'id': 2, 'text': '大象', 'subject': 'history'}"] 

In [6]:
res = client.query(
    collection_name="demo_collection",
    ids=[0, 2],
    output_fields=["vector", "text", "subject"],
)
# print(res)